<h1>Voorbeeld data import met Pandas</h1>
<h4>Door B. Braam</h4>

<h2>Imports</h2>
<p>We gaan hiermee de databases in archive gebruiken en we gaan daarvan de gegevens transformeren.
Ik begin met de juiste imports.</p>

In [1]:
import pandas as pd

Nu gaan we met behulp van pandas een dataframe aanmaken en daar de csv appearances.csv inladen.<br/>
Vervolgens controleren we de bovenste 5 rijen in de csv (de table head.)
<br/>
<br/>Dit doen we zodat we geen rijen in onze database zelf aanpassen, maar alleen ons gemaakte dataframe.

In [2]:
df = pd.read_csv('archive/appearances.csv')
print(df.head())

    appearance_id  game_id  player_id  player_club_id  player_current_club_id  \
0  2460589_237947  2460589     237947           11411                   16247   
1  2460503_150846  2460503     150846            4807                    2503   
2  2460507_136142  2460507     136142             271                     862   
3  2460508_153735  2460508     153735           18506                    4128   
4   2460509_27017  2460509      27017           10482                    3719   

         date       player_name competition_id  yellow_cards  red_cards  \
0  2014-07-01  Rumyan Hovsepyan            CLQ             0          0   
1  2014-07-03           Yoro Ly            ELQ             0          0   
2  2014-07-03   Armando Vajushi            ELQ             0          0   
3  2014-07-03  Marko Simonovski            ELQ             0          0   
4  2014-07-03   Dmitriy Khomich            ELQ             0          0   

   goals  assists  minutes_played  
0      0        0         

Vervolgens gaan we de dataframe verkleinen zodat we kunnen kijken hoeveel rode kaarten elke speler heeft. Wat we hiervoor nodig hebben is player_id en red_cards.

In [3]:
dfRedCards = df.loc[:,['player_id','red_cards']]
print(dfRedCards.head())

   player_id  red_cards
0     237947          0
1     150846          0
2     136142          0
3     153735          0
4      27017          0


Nu gaan we doen wat in SQL werkt als " Group By". Wat dit betekent is dat we alle rode kaarten in alle potjes per speler bij elkaar optellen. Dus bijvoorbeeld, als Lionel Messi in 2 toernooien 2 rode kaarten heeft, worden die bij elkaar opgeteld. Momenteel staat het nog per wedstrijd zelf.
<br/><br/>
group by player_id, en vat red_cards samen als de som van alle red_cards die die speler heeft gehaald.
<br/>met reset_index() wijzen we opnieuw rij-labels toe (1,2,3,...) zodat de dataframe leesbaar is mocht je die tussendoor willen uitlezen.

In [4]:
dfRedCards = dfRedCards.groupby('player_id')['red_cards'].sum().reset_index()
print(dfRedCards.head())

   player_id  red_cards
0         10          0
1         26          0
2         65          1
3         80          0
4        132          0


Nu gaan we dit sorteren op meeste rode kaarten naar laagste.<br/>
In jupyter notebooks kan je gewoon een variabelenaam in zijn eentje op een regel zetten en print de IDE de value van die variabele.

In [6]:
dfRedCards.sort_values(by=['red_cards'], ascending=False)
dfRedCards 

,player_id,red_cards
0,10,0
1,26,0
2,65,1
3,80,0
4,132,0
...,...,...
20103,1141487,0
20104,1143805,0
20105,1143809,0
20106,1143811,0


Vervolgens behouden we alleen nog maar de rijen waar het aantal red_cards niet gelijk is aan 0 (en verwijderen we die uit de dataframe.)
Hiervoor wil ik eerst controleren wat het datatype is van red_cards zodat ik geen bewerkingen op int uit probeer te voeren als het een string is.

In [9]:
dfRedCards.dtypes

player_id    int64
red_cards    int64
dtype: object

Mooi! red_cards is een int.

In [10]:
dfRedCards = dfRedCards[dfRedCards['red_cards']!=0]
dfRedCards

,player_id,red_cards
2,65,1
21,1443,1
27,1784,1
30,1896,1
43,2587,1
...,...,...
19876,911400,1
19921,929600,1
19977,962090,1
19978,962555,1


<h2>Tabel combineren met andere tabel</h2>
<p>Nu gaan we wat doen dat lijkt op JOIN ON in SQL. We gaan de tabel met player_id en hun naam importeren, en aan deze tabel plakken. Vervolgens verwijderen we de kolom player_id in ons dataframe.</p>

In [14]:
dfSpeler = pd.read_csv('archive/players.csv')
dfGecombineerd = pd.merge(dfSpeler,dfRedCards, on="player_id")
dfGecombineerd.dtypes
dfGecombineerd = dfGecombineerd[['name','red_cards']]
dfGecombineerd

,name,red_cards
0,Kenny Miller,1
1,Fabricio Coloccini,2
2,Ricardo Costa,1
3,Wederson,1
4,Theodoros Tripotseris,1
...,...,...
3299,Albano Bizzarri,1
3300,Marco Reus,1
3301,Walace,1
3302,Daniele Padelli,1
